In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
from pandas import DataFrame
import numpy as np
import matplotlib.pyplot as plt
from data_processing.subject_selectors import *
from data_processing.window_splitters import *
from data_processing.train_test_spliters import *
import pathlib
from os.path import join, isfile
from os import listdir
import itertools
from sklearn.mixture import GaussianMixture
from sklearn.metrics import roc_auc_score
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
import math
import warnings 

In [ ]:
subjectsPath = join(pathlib.Path().resolve(), "data\\me_time_fused\\subjects")
allSubjects = [f[:-4] for f in listdir(subjectsPath) if isfile(join(subjectsPath, f))]
allSubjects

In [ ]:
x = [allSubjects[0]]

subjects: dict[str, DataFrame] = SubjectSelectorList(subjectsPath, x, True, False).get_subjects()
print(subjects)

In [ ]:
def groupSplitter(data: DataFrame, window: int, stride: int) -> tuple[list[np.array], int]:
    if window % stride != 0:
        raise Exception("window not divisible by stride")

    first = pd.to_datetime(np.array(data["time"])[0])
    first = first.replace(hour = 0, minute = 0, second = 0, microsecond = 0)

    currentOffset = 0
    groups: list[np.array] = list()

    windowInHours = window // 60
    datapointsPerGroup = window * 60 // 5

    while currentOffset < window:
        offsetTime = data["time"] - pd.Timedelta(minutes = currentOffset)
        groups.append(np.array(offsetTime.dt.hour // windowInHours + (offsetTime - first).dt.days * (24 / windowInHours)).astype(int))
        currentOffset += stride
    return groups, datapointsPerGroup


    # return np.array((data["time"] - pd.Timedelta(minutes = stride)).dt.hour + (data["time"] - first).dt.days * 24), 720

def groupSplitterPerTimeframe(data: DataFrame) -> tuple[np.array, int]:
    first = pd.to_datetime(np.array(data["time"])[0])
    first = first.replace(hour = 0, minute = 0, second = 0, microsecond = 0)
    return np.array(data["time"].dt.hour // 6 + (data["time"] - first).dt.days * 4), 4320

In [ ]:
for subject in allSubjects:
    data = SubjectSelectorList(subjectsPath, [subject], True, False).get_subjects()
    data = data[subject]
    windows = WindowSplitterWithStride().split((subject, data), 60 * 3, 20)
    windows.to_pickle(join(pathlib.Path().resolve(), "data\\me_time_fused\\windowed_subjects\\3_hour_20_stride", subject + ".pkl"))
    print(subject)

In [ ]:
def function(x):
    aux = x["time"].max() - x["time"].min()
    if aux > pd.Timedelta(minutes = 60 * 6):
        print(x["time"].min())
        print(x["time"].max())
        print()
    return x["time"].max() - x["time"].min()

for x in a:
    unique, counts = np.unique(x, return_counts=True)
    freq = np.asarray((unique, counts)).T
    elemLess = freq[~(freq == 720)[:, 1]][:, 0]
    filter = np.in1d(x, elemLess)
    x[filter] = -1
    group = data.groupby(x)
    print(group.groups.keys())
    break

In [ ]:
print("[", end = "")
count = 0
for x in allSubjects:
    subjectData: DataFrame = pd.read_pickle(join(pathlib.Path().resolve(), "data\\me_time_fused\\windowed_subjects\\1_hour_no_stride", x + ".pkl")) 
    if len(subjectData) > 1500:
        print("\"" + x + "\",", end = "")
        count += 1
print("]")

print(count)
    

In [ ]:
from models.transformers import *

data = np.array([[1,2,3,4,5,6,7,8], [4,5,6,4,5,6,7,8], [4,5,6,4,5,6,7,8]]).astype(float)

transformer = Transformer(transformations = [np.mean, np.median], pca_components = 2, mfcc_components = 2)

transformer.fit(data).transform(data)